# Workflow Debugging

In [ ]:
from predictor.driver import Predictor
from predictor.src import utils

import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split


(
    training_data,
    names_list,
    scores_list,
) = Predictor.generate_trainingdata_from_qasm_files(
    folder_path="qasm_files/"  # , compiled_path="qasm_compiled_postprocessed/"
)

In [ ]:
X, y = zip(*training_data)
Predictor.train_decision_tree_classifier(X, y, names_list, scores_list);

In [ ]:
from numpy import asarray
from numpy import save

data = asarray(training_data)
save("training_data.npy", data)
data = asarray(names_list)
save("names_list.npy", data)
data = asarray(scores_list)
save("scores_list.npy", data)

In [ ]:
from numpy import asarray
from numpy import save
from predictor.driver import Predictor
from predictor.src import utils

import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

training_data = np.load("training_data.npy", allow_pickle=True)

names_list = list(np.load("names_list.npy", allow_pickle=True))

scores_list = list(np.load("scores_list.npy", allow_pickle=True))
X, y = zip(*training_data)
X = list(X)
y = list(y)
for i in range(len(X)):
    X[i] = list(X[i])
    scores_list[i] = list(scores_list[i])

In [ ]:
Predictor.train_decision_tree_classifier(X, y, names_list, scores_list);

# Decision Path Generation

In [ ]:
names_list[30]

In [ ]:
non_zero_indices = []
X = np.array(X)
y = np.array(y)
for i in range(len(X[0])):
    if sum(X[:, i]) > 0:
        non_zero_indices.append(i)
X = X[:, non_zero_indices]

openqasm_qc_list = utils.get_openqasm_gates()
res = [openqasm_qc_list[i] for i in range(0, len(openqasm_qc_list))]
res.append("num_qubits")
res.append("depth")
for i in range(1, 6):
    res.append(str(i) + "_max_interactions")

res = [res[i] for i in non_zero_indices]
index = 30
X_test = X[[index]]

clf_instance = Predictor._clf.best_estimator_
n_nodes = clf_instance.tree_.node_count
children_left = clf_instance.tree_.children_left
children_right = clf_instance.tree_.children_right
feature = clf_instance.tree_.feature
threshold = clf_instance.tree_.threshold
node_indicator = clf_instance.decision_path(X_test)
leaf_id = clf_instance.apply(X_test)

sample_id = 0
# obtain ids of the nodes `sample_id` goes through, i.e., row `sample_id`
node_index = node_indicator.indices[
    node_indicator.indptr[sample_id] : node_indicator.indptr[sample_id + 1]
]

print("Rules used to predict sample {id}:\n".format(id=sample_id))
for node_id in node_index:
    # continue to the next node if it is a leaf node
    if leaf_id[sample_id] == node_id:
        continue
    # check if value of the split feature for sample 0 is below threshold
    if X_test[sample_id, feature[node_id]] <= threshold[node_id]:
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print(
        "decision node {node} : (X_test[{sample}, {feature}] = {value}) "
        "{inequality} {threshold})".format(
            node=node_id,
            sample=sample_id,
            feature=res[feature[node_id]],
            value=X_test[sample_id, feature[node_id]],
            inequality=threshold_sign,
            threshold=threshold[node_id],
        )
    )
print("Prediction: ", clf_instance.predict(X_test))

In [ ]:
(
    X_train,
    X_test,
    y_train,
    y_test,
) = train_test_split(X, y, test_size=0.3, random_state=5)
for i in range(19):
    print(y_test.count(i), utils.get_machines()[i])

In [ ]:
print(node_indicator)

# Debugging

In [ ]:
machines = utils.get_machines()
[machines[i] for i in list(Predictor._clf.classes_)]

In [ ]:
from qiskit import QuantumCircuit
from mqt.bench import get_one_benchmark

qc = get_one_benchmark("ghz", 1, 5)

# Nachbehandlung OQC qasm files

In [ ]:
import os
from qiskit import QuantumCircuit

directory = "qasm_compiled"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if "oqc_qiskit" in f:
        with open(f, "r") as f:
            lines = f.readlines()
        new_name = os.path.join("qasm_compiled_postprocessed", filename)
        with open(new_name, "w") as f:
            for line in lines:
                if not (
                    "gate rzx" in line.strip("\n") or "gate ecr" in line.strip("\n")
                ):
                    f.write(line)
                if "gate ecr" in line.strip("\n"):
                    f.write(
                        "gate rzx(param0) q0,q1 { h q1; cx q0,q1; rz(param0) q1; cx q0,q1; h q1; }\n"
                    )
                    f.write(
                        "gate ecr q0,q1 { rzx(pi/4) q0,q1; x q0; rzx(-pi/4) q0,q1; }\n"
                    )

        qc = QuantumCircuit.from_qasm_file(new_name)
        print("New qasm file for: ", new_name)

    elif "oqc_tket" in f:
        with open(f, "r") as f:
            lines = f.readlines()
        new_name = os.path.join("qasm_compiled_postprocessed", filename)
        with open(new_name, "w") as f:
            count = 0
            for line in lines:
                f.write(line)
                count += 1
                if count == 2:
                    f.write(
                        "gate rzx(param0) q0,q1 { h q1; cx q0,q1; rz(param0) q1; cx q0,q1; h q1; }\n"
                    )
                    f.write(
                        "gate ecr q0,q1 { rzx(pi/4) q0,q1; x q0; rzx(-pi/4) q0,q1; }\n"
                    )
        qc = QuantumCircuit.from_qasm_file(new_name)
        print("New qasm file for: ", new_name)

# Nachbehandlung: Check if File is readable at all

In [ ]:
import os
from qiskit import QuantumCircuit

directory = "qasm_compiled"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if f.endswith(".qasm"):
        try:
            qc = QuantumCircuit.from_qasm_file(f)
        except Exception as e:
            print("Fail: ", e)
            # Delete file: os.remove(f)

In [ ]:
import glob

files = glob.glob("qasm_compiled/ae_indep_2_*")
print(files)

In [ ]:
np.median([i for i in range(1, 20)])

# Debugging MQT Bench Integration

In [1]:
from predictor.driver import Predictor
from mqt.bench import get_one_benchmark
from qiskit import QuantumCircuit

qc = get_one_benchmark("dj", 1, 3)
QuantumCircuit.qasm(qc, filename="test.qasm")
Predictor.compile_all_circuits_for_qc("test.qasm", "./comp_test")

./comp_test/test0.qasm
./comp_test/test1.qasm
./comp_test/test2.qasm
./comp_test/test3.qasm
./comp_test/test4.qasm
./comp_test/test5.qasm
./comp_test/test6.qasm
./comp_test/test7.qasm
./comp_test/test8.qasm
./comp_test/test9.qasm
./comp_test/test10.qasm
./comp_test/test11.qasm
./comp_test/test12.qasm
./comp_test/test13.qasm
./comp_test/test14.qasm
./comp_test/test15.qasm
./comp_test/test16.qasm
./comp_test/test17.qasm
./comp_test/test18.qasm
./comp_test/test19.qasm
./comp_test/test20.qasm
./comp_test/test21.qasm
./comp_test/test22.qasm
./comp_test/test23.qasm
./comp_test/test24.qasm
./comp_test/test25.qasm
./comp_test/test26.qasm
./comp_test/test27.qasm
./comp_test/test28.qasm
./comp_test/test29.qasm


True

In [ ]:
from distutils.sysconfig import get_python_lib

print(get_python_lib())